# Figure S1E
## coverage stats for synthetic influenza data

#### Written by: Kate Johnson

In [1]:
# load necessary packages
suppressMessages({
library('plyr')
library('dplyr')
library('tidyverse')
library('ggplot2')
library('glue')
library("stringr")
})


Attaching package: ‘dplyr’


The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching packages ──────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0     ✔ purrr   0.3.5
✔ tibble  3.1.8     ✔ stringr 1.5.0
✔ tidyr   1.2.1     ✔ forcats 0.5.2
✔ readr   2.1.3     
── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::arrange()   masks plyr::arrange()
✖ purrr::compact()   masks plyr::compact()
✖ dplyr::count()     masks plyr::count()
✖ dplyr::failwith()  masks plyr::failwith()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::id()        masks

In [2]:
message("set working directory and file names")

wkdir = "../synthetic_check/"

covreport = 'cov_data.tsv' # h1n1

af_report = 'flu.synthetic.afdata.csv' # h1n1


set working directory and file names



In [3]:
source(glue('{wkdir}/../scripts/maf_functions.R'))  # USER WILL NEED TO UPDATE PATH TO FUNCTIONS

In [4]:
message("Set working directory and read in sim data")

setwd(wkdir) 

cov = read.csv(covreport,header = T, sep = '\t')

cov$source_ID = str_replace(cov$name, '.fas_BWA', '')

af = read.csv(af_report,header = T) %>%
    select(sample, source_ID, copy_number, Rep) %>%
    unique()

af$MIX = str_extract(string = af$source_ID, pattern = "Mix[0-9]+")

# removing bad samples from analysis
remove_mix = c("Mix37", "Mix38", "Mix39", "Mix40", "Mix41", "Mix42", "Mix44")

levels(factor((af %>% filter(MIX %in% remove_mix))$sample))

sample_remove= c('0.0039_10^3_rep2',
                  '0.0078_10^3_rep2',
                  '0.0156_10^3_rep2',
                  '0.0313_10^3_rep2',
                  '0.0625_10^7_rep1',
                  '0.25_10^7_rep1',
                  '0.5_10^7_rep1',
                '0.5_10^3_rep2',
                '0.5_10^3_rep1')




Set working directory and read in sim data



[1] "0.0039_10^3_rep1" "0.0078_10^3_rep1" "0.0156_10^3_rep1" "0.0313_10^3_rep1"
[5] "0.0625_10^7_rep2" "0.25_10^7_rep2"   "0.5_10^7_rep2"

In [5]:
# min and max coverage across the positions/segments/samples: 
# filter data
grab_source_ids = (af %>% filter( !sample %in% sample_remove &
                    !MIX %in% remove_mix & 
                   copy_number != '10^7' ) %>% unique())$source_ID
# grab min and max
min((cov %>% filter(source_ID %in% grab_source_ids) %>% unique())$totalcount)
max((cov %>% filter(source_ID %in% grab_source_ids) %>% unique())$totalcount)

[1] 291

[1] 36983

In [6]:
cov = merge(cov, af, by = c('source_ID')) %>%
    filter( !sample %in% sample_remove &
                    !MIX %in% remove_mix & 
                   copy_number != '10^7' ) %>%
    unique() #filter and merge with metadata to make plots


# Figure S1E: Coverage of synthetic influenza data

In [ ]:
cov$segment = factor(cov$segment, levels = c('PB2','HA','NA'))

plots1e = ggplot(cov, aes(x=ntpos, y = log10(totalcount), color = factor(copy_number), group = source_ID)) + 
    geom_line() + 
    facet_grid(.~segment, scales ='free_x') +
    geom_hline(yintercept = log10(1000), linetype = 2, color = 'black') +
    geom_hline(yintercept = log10(10000), linetype = 2, color = 'black') +
    scale_color_brewer(palette = 'Paired') +
    labs(y='read depth (log10)', x= 'nucleotide position', color = 'copy number') + 
    PlotTheme1

print(plots1e)
ggsave(plots1e,
       filename = glue("{wkdir}/figS1E_kj.png"),
       width = 10,
       height = 4, limitsize=FALSE)

ggsave(plots1e,
       filename = glue("{wkdir}/figS1E_kj.pdf"),
       width = ,
       height = 4, limitsize=FALSE, useDingbats = FALSE)



In [ ]:
af = read.csv(af_report,header = T) %>%
    select(sample, segment,source_ID, ntpos, cat) %>%
    unique()
af$SEGMENT = paste0('H1N1_', af$segment)
colnames(af)

af$MIX = str_extract(string = af$source_ID, pattern = "Mix[0-9]+")

head(af)
colnames(af)

cov2 = merge(cov, af, by = c('source_ID','ntpos'), all.x =TRUE) %>%
    unique()


In [ ]:
# read depth is higher in raw mpileup output
cov2 %>% 
    group_by(cat) %>%
    mutate(median_cov = median(totalcount),
          mean_cov = mean(totalcount)) %>%
    select(cat, median_cov, mean_cov) %>%
    unique()

In [ ]:
mean_cov_across = cov %>%
            group_by(segment, ntpos) %>%
            mutate(mean_totalcount = mean(totalcount)) %>%
            ungroup() %>%
            select(segment, ntpos, mean_totalcount)

In [ ]:
length(levels(factor(cov$source_ID)))
quantile(cov$totalcount)
quantile(cov$totalcount)[2][[1]]
quantile(cov$totalcount)[4][[1]]

In [ ]:
mean_cov = cov %>% 
            group_by(segment,copy_number) %>%
            mutate(mean_coverage = mean(totalcount),
                  sd_coverage = sd(totalcount),
                  median_coverage = median(totalcount), 
                  coverage_IQR = IQR(totalcount),
                  min_coverage = min(totalcount),
                  max_coverage = max(totalcount),
                  first_quart = quantile(totalcount)[2][[1]],
                  third_quart = quantile(totalcount)[4][[1]]) %>%
            ungroup() %>%
            select(segment, copy_number,
                  mean_coverage, sd_coverage, median_coverage,
                  coverage_IQR, min_coverage, max_coverage,
                  first_quart, third_quart) %>%
            unique()


ggplot(mean_cov, aes(x=copy_number, y = mean_coverage, color =  copy_number)) + 
    
    geom_errorbar(aes(ymin=mean_coverage - sd_coverage,
                      ymax=mean_coverage + sd_coverage), width = 0.2) +
    geom_point(size = 3) +
    PlotTheme1 + 
    ylim(0, 15000) +
    scale_color_brewer(palette = 'Paired') + 
    facet_grid(.~segment)




In [ ]:
cov %>% 
            group_by(segment) %>%
            mutate(mean_coverage = mean(totalcount),
                  sd_coverage = sd(totalcount),
                  median_coverage = median(totalcount), 
                  coverage_IQR = IQR(totalcount),
                  min_coverage = min(totalcount),
                  max_coverage = max(totalcount),
                  first_quart = quantile(totalcount)[2][[1]],
                  third_quart = quantile(totalcount)[4][[1]]) %>%
            ungroup() %>%
            select(segment,
                  mean_coverage, sd_coverage, median_coverage,
                  coverage_IQR, min_coverage, max_coverage,
                  first_quart, third_quart) %>%
            unique()

In [ ]:
sessionInfo()